# 04. Johnson's Algorithm APSP Benchmark

This notebook benchmarks the following variants of Johnson's algorithm for All-Pairs Shortest Path (APSP):
- `johnson_serial`
- `johnson_openmp`
- `johnson_cuda`
- `johnson_hybrid`

Johnson's algorithm is efficient for sparse graphs and can handle negative edge weights, but it will fail if the graph contains a negative-weight cycle.

## 1. Setup

Copy and paste the utility functions from `00_setup_build.ipynb`.

In [ ]:
import subprocess, statistics, re, os, json, time, pandas as pd

def run_command(cmd, timeout=300):
    try:
        print("  >", cmd)
        return subprocess.run(cmd, shell=True, capture_output=True,
                             text=True, check=True, timeout=timeout).stdout
    except subprocess.CalledProcessError as e:
        print("    stderr:", e.stderr.strip())
    except subprocess.TimeoutExpired:
        print("    timeout")
    return None

def parse_time(out):
    if not out: return None
    if "negative cycle" in out.lower():
        print("    Negative cycle detected, skipping.")
        return None
    m = re.search(r"time:\s*([0-9]*\\.?[0-9]+)\\s*(ms|s|sec|seconds)?", out, re.I)
    if not m: return None
    val = float(m.group(1)); unit = (m.group(2) or "s").lower()
    return val/1000.0 if unit.startswith("ms") else val

def time_exe(cmd, warmups=1, runs=3):
    if not cmd: return None
    for _ in range(warmups): _ = run_command(cmd)
    samples = []
    for _ in range(runs):
        t = parse_time(run_command(cmd))
        if t is not None: samples.append(t)
    return statistics.median(samples) if samples else None

## 2. Dataset Selection

In [ ]:
#@markdown **Dataset Selection** for Johnson's Algorithm
use_real_data = False  #@param {type:"boolean"}
real_graph_url = "https://snap.stanford.edu/data/wiki-Vote.txt.gz"  # example for a sparse graph
graph_path = "johnson_graph.txt"

if use_real_data:
    # Note: This is a placeholder. The executable would need to be modified
    # to read a specific file format (e.g., edge list).
    print("Real data mode is selected, but requires modifying the C++ executable to load from a file.")
else:
    print("Using synthetic random graphs for Johnson's algorithm.")

## 3. Benchmark Parameters

In [ ]:
#@markdown ### Benchmark Parameters (Johnson APSP)
V_list = "50,100,200,300"      #@param {type:"string"}
min_w = -10                    #@param {type:"integer"}
max_w = 50                     #@param {type:"integer"}
density = 0.1                  #@param {type:"number"}
threads = 8                    #@param {type:"integer"}
split_ratio = 0.5              #@param {type:"number"}

V_list = [int(x) for x in V_list.split(",")]
executables = ['johnson_serial','johnson_openmp','johnson_cuda','johnson_hybrid']

### Parallelizing Johnson's Algorithm

Johnson's algorithm consists of two main phases:
1.  A single run of Bellman-Ford to compute vertex potentials for re-weighting.
2.  `V` independent runs of Dijkstra's algorithm, one for each vertex.

Phase 2 is **embarrassingly parallel**. Each Dijkstra run is independent of the others, making it a perfect candidate for parallelization. Our OpenMP implementation can leverage this by assigning different source vertices to different threads, for example, using a `#pragma omp parallel for` loop over the vertices.

## 4. Command Builder

In [ ]:
def build_cmd_johnson(exe, v, *, min_w, max_w, density, threads, split_ratio):
    path = os.path.join("bin", exe)
    if not os.path.exists(path): return None
    args = [str(v), str(min_w), str(max_w), str(density)]
    if "hybrid" in exe:
        args.insert(3, str(split_ratio))
    if ("openmp" in exe) or ("hybrid" in exe):
        args.append(str(threads))
    return " ".join([path] + args)

## 5. Run Benchmarks

In [ ]:
rows = []
for v in V_list:
    print(f"\nJohnson for V={v}")
    row = {"vertices": v}
    for exe in executables:
        cmd = build_cmd_johnson(exe, v, min_w=min_w, max_w=max_w,
                                density=density, threads=threads, split_ratio=split_ratio)
        t = time_exe(cmd)
        row[exe] = t
        if t is not None: print(f"  {exe}: {t:.6f}s")
    rows.append(row)

import pandas as pd
df_j = pd.DataFrame(rows).set_index("vertices").sort_index()
df_j.to_csv("johnson_times.csv")
df_j

## 6. Speedup Analysis

In [ ]:
import numpy as np, seaborn as sns, matplotlib.pyplot as plt
base = df_j['johnson_serial']
speed = pd.DataFrame({
    "johnson_openmp_speedup": base / df_j['johnson_openmp'],
    "johnson_cuda_speedup":   base / df_j['johnson_cuda'],
    "johnson_hybrid_speedup": base / df_j['johnson_hybrid'],
}, index=df_j.index)

display(speed)
sns.lineplot(data=speed.reset_index().melt("vertices", var_name="variant", value_name="speedup"),
             x="vertices", y="speedup", hue="variant", marker="o")
plt.axhline(1, ls="--", c="gray"); plt.yscale("log"); plt.show()
speed.to_csv("johnson_speedup.csv")

## 7. Johnson vs. Floyd-Warshall Discussion

The choice between Johnson's algorithm and Floyd-Warshall for APSP depends heavily on the graph's density.

- **Johnson's Algorithm**: `O(V*E + V²*logV)`. It excels on **sparse graphs** where `E` is much smaller than `V²`. The initial Bellman-Ford run `O(V*E)` is the main bottleneck on sparse graphs.
- **Floyd-Warshall**: `O(V³)` It is generally better for **dense graphs**, where `E` is close to `V²`. In this case, the `O(V*E)` part of Johnson's algorithm approaches `O(V³)`, making it less efficient than the more straightforward, cache-friendly Floyd-Warshall.